In [ ]:
from unsloth import FastLanguageModel
import torch

print(torch.__version__)  # should be 2.1.2

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
2.7.0+cu126


In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

df_testing_64 = pd.read_csv("/content/drive/My Drive/testing_data_64.csv")
df_context_48 = pd.read_csv("/content/drive/My Drive/context_data_48.csv")

Mounted at /content/drive


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
FastLanguageModel.for_inference(model)

def create_few_shot_prompt(text, prompting_data, N=0):

    # Base instruction
    prompt = (
        "Analyze the following text and predict the most likely MBTI (Myers-Briggs Type Indicator) personality type.\n"
        "Provide a detailed reasoning for your prediction."
    )

    # Add few-shot examples
    if N > 0:
        prompt += "\n\nHere are some examples to guide your prediction:"
        for i, row in prompting_data.head(N).iterrows():
            prompt += f"""

Example {i + 1}:
Text: {row['tweet']}
MBTI Type: {row['type']}"""

        prompt += "\n\nNow, analyze the following text:"

    # Add the input text to analyze
    prompt += f"""

### Text:
{text.strip()}

### MBTI Type Prediction:
### Reasoning:
"""

    return prompt


In [ ]:
prompt = create_few_shot_prompt("Yes and due to it being shapes and objects that means our process of analyzing is not so much bound by languages", df_context_48, N=0)
inputs = tokenizer([prompt], return_tensors="pt", truncation=True, max_length=2048).to("cuda")
outputs = model.generate(**inputs, max_new_tokens=512)

decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(decoded)

Analyze the following text and predict the most likely MBTI (Myers-Briggs Type Indicator) personality type.
Provide a detailed reasoning for your prediction.

### Text:
Yes and due to it being shapes and objects that means our process of analyzing is not so much bound by languages

### MBTI Type Prediction:
### Reasoning:
The text is written in a way that it is easy to understand. It is clear that the speaker is not bound by language when analyzing the shapes and objects. This means that the speaker is not limited by their own language or cultural biases when analyzing the shapes and objects. This is a sign of a Judging personality type, which is the opposite of the Perceiving personality type. The speaker is also not concerned with the rules and regulations of the shapes and objects, which is a sign of a Judging personality type. The speaker is also not concerned with the rules and regulations of the shapes and objects, which is a sign of a Judging personality type. The speaker is als

In [ ]:
FastLanguageModel.for_inference(model)

print('N=48')
n_samples = 48
results = []

for idx, row in df_testing_64.head(2).iterrows():
  text = row['tweet']
  actual_type = row['type']

  prompt = create_few_shot_prompt(text, df_context_48, N=n_samples)
  inputs = tokenizer([prompt], return_tensors="pt", truncation=True, max_length=2048).to("cuda")
  outputs = model.generate(**inputs, max_new_tokens=512)

  decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

  if "### MBTI Type Prediction:" in decoded:
        response = decoded.split("### MBTI Type Prediction:")[-1].strip()
  else:
        response = decoded

  results.append({
      "text": text,
      "actual_type": actual_type,
      "response": response
  })

  print(f"Text: {text}")
  print(f"Response: {response}")
  print(f"Processed row {idx + 1}/{len(df_testing_64)}")
  print("----------------------------------------------")

with open('llama_N48.json', 'w') as f:
    json.dump(results, f, indent=4)

print("MBTI predictions saved to 'llama_N48.json'.")

N=48
Text: Yes and due to it being shapes and objects that means our process of analyzing is not so much bound by languages
Response: Analyze the following text and predict the most likely MBTI (Myers-Briggs Type Indicator) personality type.
Provide a detailed reasoning for your prediction.

Here are some examples to guide your prediction:

Example 1:
Text: everything in life is always amazing to me i just loove laughing and keeping a positive enviourmentcrazy
MBTI Type: ENFJ

Example 2:
Text: Shootmepis definitely has depression or something similar When someone is feeling truly depressed they feel completely dead inside and they feel like they can never get better I hope that he or she attempts to get help from a mental professional
MBTI Type: ENFJ

Example 3:
Text: exactly i think it is all tendencies i am somewhat of a cancerleo cusp and i can tell that i am influenced by that a little bit as well not as much a personality type leos are very passionate
MBTI Type: ENFJ

Example 4:
T